In [1]:
from datetime import date
import exportdata

In [2]:
df = exportdata.extract_df()
df_smoothed = exportdata.add_weekend(df)

In [3]:
display(df)
display(df_smoothed)

,staff.on,staff.off,student.on,student.off,staff7.on,staff7.off,student7.on,student7.off,stafftotal.on,stafftotal.off,studenttotal.on,studenttotal.off
date,,,,,,,,,,,,
2020-10-09,1.0,1.0,27.0,0.0,3.0,1.0,64.0,17.0,6.0,3.0,72.0,17.0
2020-10-12,1.0,0.0,54.0,0.0,3.0,1.0,112.0,6.0,7.0,3.0,126.0,17.0
2020-10-13,1.0,0.0,12.0,7.0,5.0,2.0,117.0,12.0,8.0,3.0,138.0,24.0
2020-10-14,1.0,1.0,21.0,7.0,6.0,3.0,129.0,17.0,9.0,4.0,159.0,31.0
2020-10-15,0.0,1.0,14.0,5.0,4.0,4.0,122.0,19.0,9.0,5.0,173.0,36.0
2020-10-16,0.0,0.0,8.0,8.0,2.0,4.0,114.0,20.0,9.0,5.0,181.0,44.0
2020-10-19,3.0,1.0,33.0,8.0,4.0,3.0,140.0,24.0,12.0,6.0,214.0,52.0
2020-10-20,1.0,1.0,21.0,4.0,4.0,3.0,138.0,24.0,13.0,7.0,235.0,56.0
2020-10-21,2.0,2.0,11.0,4.0,6.0,4.0,136.0,25.0,15.0,9.0,246.0,60.0


,staff.on,staff.off,student.on,student.off,staff7.on,staff7.off,student7.on,student7.off,stafftotal.on,stafftotal.off,studenttotal.on,studenttotal.off
date,,,,,,,,,,,,
2020-10-09,1.000000,1.000000,27.0,0.000000,3.0,1.0,64.0,17.0,6.0,3.0,72.0,17.0
2020-10-10,0.333333,0.000000,18.0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-10-11,0.333333,0.000000,18.0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-10-12,0.333333,0.000000,18.0,0.000000,3.0,1.0,112.0,6.0,7.0,3.0,126.0,17.0
2020-10-13,1.000000,0.000000,12.0,7.000000,5.0,2.0,117.0,12.0,8.0,3.0,138.0,24.0
2020-10-14,1.000000,1.000000,21.0,7.000000,6.0,3.0,129.0,17.0,9.0,4.0,159.0,31.0
2020-10-15,0.000000,1.000000,14.0,5.000000,4.0,4.0,122.0,19.0,9.0,5.0,173.0,36.0
2020-10-16,0.000000,0.000000,8.0,8.000000,2.0,4.0,114.0,20.0,9.0,5.0,181.0,44.0
2020-10-17,1.000000,0.333333,11.0,2.666667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.loc[:,["staff7.on","staff7.off","student7.on","student7.off"]]

,staff7.on,staff7.off,student7.on,student7.off
date,,,,
2020-10-09,3.0,1.0,64.0,17.0
2020-10-12,3.0,1.0,112.0,6.0
2020-10-13,5.0,2.0,117.0,12.0
2020-10-14,6.0,3.0,129.0,17.0
2020-10-15,4.0,4.0,122.0,19.0
2020-10-16,2.0,4.0,114.0,20.0
2020-10-19,4.0,3.0,140.0,24.0
2020-10-20,4.0,3.0,138.0,24.0
2020-10-21,6.0,4.0,136.0,25.0


In [5]:
f=df.loc[:,["stafftotal.on","stafftotal.off","studenttotal.on","studenttotal.off"]].diff().dropna()
g=df.loc[:,["staff7.on","staff7.off","student7.on","student7.off"]].diff().dropna()
g

,staff7.on,staff7.off,student7.on,student7.off
date,,,,
2020-10-12,0.0,0.0,48.0,-11.0
2020-10-13,2.0,1.0,5.0,6.0
2020-10-14,1.0,1.0,12.0,5.0
2020-10-15,-2.0,1.0,-7.0,2.0
2020-10-16,-2.0,0.0,-8.0,1.0
2020-10-19,2.0,-1.0,26.0,4.0
2020-10-20,0.0,0.0,-2.0,0.0
2020-10-21,2.0,1.0,-2.0,1.0
2020-10-22,0.0,0.0,4.0,-1.0


In [6]:
df.loc[date(2020,10,12):,["staff.on","staff.off","student.on","student.off"]].sub(f.values)

,staff.on,staff.off,student.on,student.off
date,,,,
2020-10-12,0.0,0.0,0.0,0.0
2020-10-13,0.0,0.0,0.0,0.0
2020-10-14,0.0,0.0,0.0,0.0
2020-10-15,0.0,0.0,0.0,0.0
2020-10-16,0.0,0.0,0.0,0.0
2020-10-19,0.0,0.0,0.0,0.0
2020-10-20,0.0,0.0,0.0,0.0
2020-10-21,0.0,0.0,0.0,0.0
2020-10-22,0.0,0.0,0.0,0.0


In [7]:
rolling=df.loc[:,["staff.on","staff.off","student.on","student.off"]].rolling(5).sum().dropna()

In [8]:
reported=df.loc[date(2020,10,15):,["staff7.on","staff7.off","student7.on","student7.off"]]

In [9]:
reported.sub(rolling.values)

,staff7.on,staff7.off,student7.on,student7.off
date,,,,
2020-10-15,0.0,1.0,-6.0,0.0
2020-10-16,-1.0,2.0,5.0,-7.0
2020-10-19,-1.0,0.0,52.0,-11.0
2020-10-20,-1.0,-1.0,41.0,-8.0
2020-10-21,0.0,-1.0,49.0,-4.0
2020-10-22,0.0,0.0,52.0,-6.0


In [10]:
rolling

,staff.on,staff.off,student.on,student.off
date,,,,
2020-10-15,4.0,3.0,128.0,19.0
2020-10-16,3.0,2.0,109.0,27.0
2020-10-19,5.0,3.0,88.0,35.0
2020-10-20,5.0,4.0,97.0,32.0
2020-10-21,6.0,5.0,87.0,29.0
2020-10-22,6.0,4.0,88.0,30.0


In [11]:
df

,staff.on,staff.off,student.on,student.off,staff7.on,staff7.off,student7.on,student7.off,stafftotal.on,stafftotal.off,studenttotal.on,studenttotal.off
date,,,,,,,,,,,,
2020-10-09,1.0,1.0,27.0,0.0,3.0,1.0,64.0,17.0,6.0,3.0,72.0,17.0
2020-10-12,1.0,0.0,54.0,0.0,3.0,1.0,112.0,6.0,7.0,3.0,126.0,17.0
2020-10-13,1.0,0.0,12.0,7.0,5.0,2.0,117.0,12.0,8.0,3.0,138.0,24.0
2020-10-14,1.0,1.0,21.0,7.0,6.0,3.0,129.0,17.0,9.0,4.0,159.0,31.0
2020-10-15,0.0,1.0,14.0,5.0,4.0,4.0,122.0,19.0,9.0,5.0,173.0,36.0
2020-10-16,0.0,0.0,8.0,8.0,2.0,4.0,114.0,20.0,9.0,5.0,181.0,44.0
2020-10-19,3.0,1.0,33.0,8.0,4.0,3.0,140.0,24.0,12.0,6.0,214.0,52.0
2020-10-20,1.0,1.0,21.0,4.0,4.0,3.0,138.0,24.0,13.0,7.0,235.0,56.0
2020-10-21,2.0,2.0,11.0,4.0,6.0,4.0,136.0,25.0,15.0,9.0,246.0,60.0


In [12]:
df.loc[:,["student7.on"]].mean()/7

student7.on    17.314286
dtype: float64

In [13]:
df.loc[:,["student.on"]].mean()

student.on    21.6
dtype: float64

In [14]:
rolling

,staff.on,staff.off,student.on,student.off
date,,,,
2020-10-15,4.0,3.0,128.0,19.0
2020-10-16,3.0,2.0,109.0,27.0
2020-10-19,5.0,3.0,88.0,35.0
2020-10-20,5.0,4.0,97.0,32.0
2020-10-21,6.0,5.0,87.0,29.0
2020-10-22,6.0,4.0,88.0,30.0


In [15]:
reported

,staff7.on,staff7.off,student7.on,student7.off
date,,,,
2020-10-15,4.0,4.0,122.0,19.0
2020-10-16,2.0,4.0,114.0,20.0
2020-10-19,4.0,3.0,140.0,24.0
2020-10-20,4.0,3.0,138.0,24.0
2020-10-21,6.0,4.0,136.0,25.0
2020-10-22,6.0,4.0,140.0,24.0
